# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import numpy as np

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [109]:
# names us
def df_names_us():
    path = "us_data/"
    filenames = []
    for i in range(2021-1879):
        filenames.append(path + "yob" + str(1880 + i) + ".txt")
    single_dfs = []
    i = 0
    for filename in filenames:
        colnames=['name', 'gender', 'births'] 
        df = pd.read_csv(filename, names=colnames, header=None)
        #year = (1880 + i) * np.ones(df.shape[0])
        df.insert(loc = 0, column = "year", value = (1880 + i))
        single_dfs.append(df)
        i += 1
    #df = pd.DataFrame()
    big_df = pd.concat(single_dfs, ignore_index=True)
    return big_df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [110]:
# names fr
def df_names_fr():
    df = pd.read_csv("fr_data/nat2021.csv",sep=';')
    df['sexe'] = df['sexe'].apply(lambda x: 'M' if (x==1) else 'F')
    df = df[['annais', 'preusuel', 'sexe', 'nombre']]
    df = df.rename(columns={"annais": "year", "preusuel": "name", "sexe":"gender", "nombre":"births"})
    df = df.sort_values(by = ['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    df = df[df['year'] != 'XXXX']
    df = df[df['name'] != '_PRENOMS_RARES']
    df = df[df['name'].apply(lambda x: len(str(x)) > 1)]
    df = df[df['name'].apply(lambda x: (isinstance(x,str)))]
    df['name'] = df['name'].apply(lambda x: x.capitalize())
    df['name'] = df['name'].str.replace('-','')
    df = df.reset_index()
    df = df.drop(columns=['index'])
    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

# taux de change

In [111]:
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export.csv",sep=';')
    df = df.drop([0,1, 2, 3, 4])
    df = df.rename(columns={"Titre :": "Date"})
    df2 = df
    for column_name , series in df2.iteritems():
        if any(s in column_name for s in devises):
            df = df.rename(columns={column_name: column_name[-4:-1]})
        else:
            if (column_name != "Date"):
                df = df.drop(columns=[column_name])
    #df["Date"] = df["Date"].to_datetime(format="%d/%m/%Y")
    df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")  
    df = df.sort_values(by = ["Date"], ascending=[True])
    df = df.set_index("Date")
    for column_name , series in df.iteritems():
        df = df[df[column_name] != '-']
        #df = df[column_name].apply(lambda x: float(x.replace(',','.')))
    df = df.replace(',','.',regex=True)
    df = df.astype('float')
    return df

### Tests

In [112]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648329)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [113]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [114]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.668s

OK
